# Q-learning neural nets

This notebook serves mostly as a reference for myself - to look things up if I don't remember exactly how things work. 

My previous implementation of reinfocement learning uses the Q-table approach. To get familiar with Deep-Q-learning, I reimplement it with a neural network. 

Many games contain a lot of states, which cannot be simply summarized in a table like in the Q-table approach. This is where neural networks comes in. Neural networks work as a function approximator and replace the Q table we have. 

Let's model the same game using a simple neural network that replaces the Q-table. Our neural network will take a flattened vector as input (16x1) and produce a vector of 4 Q-values, one for each action (left, down, right, up).  A neural net has the benefit that we can quickly make the neural net more complex by adding different and more layers to it. However, we cannot directly update any values like in the Q-table. We must train the neural net through backpropagation and a loss function. We can use a sum of squares loss function here, where the difference between the predicted values and target value is computed and the gradients passed through the network.  

In [130]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

### Neural network

The game _FrozenLake-v0_ has 16 states. I model this as a vector with shape (1, 16) that is to be inpupt in the neural network. Below is some experimenting in modeling the neural net and its in/outputs.

In [100]:
data = np.random.random((100, 16))
labels = np.random.randint(4, size=(100,4))

In [102]:
model = Sequential()
model.add(Dense(10, input_dim=16, activation = 'relu'))
model.add(Dense(4, activation='sigmoid'))
model.compile(optimizer='rmsprop',
                  loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 44        
Total params: 214
Trainable params: 214
Non-trainable params: 0
_________________________________________________________________


In [103]:
model.fit(data, labels)

Epoch 1/1
100/100 [==============================] - 0s 1ms/step - loss: 2.3889


The neural network works in this form

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 4)                 68        
Total params: 68
Trainable params: 68
Non-trainable params: 0
_________________________________________________________________


### Modeling the agent

In [123]:
class Agent:
    def __init__(self, environment, learning_rate, discount_rate):
        """
        Creates an instance of Agent. 
        
        :param: environment: Gym.environment an environment that comes with the gym package from OpenAI
        :param: learning_rate: float a value between 0 and 1. Not used in this implementation
        :param: discount_rate: float a value between 0 and 1. A higher value will consider future actions more heavily.
        """
        self.env = environment
        self.alpha = learning_rate
        self.gamma = discount_rate
#         self.q_table = np.zeros((env.observation_space.n, env.action_space.n ))
        self.q = self._create_neural_net()
        
    def _create_neural_net(self):
        """
        Creates a neural network. The input parameters of this network are designed for the 
        environment "Frozenlake-v0".
        """
        model = Sequential()
        model.add(Dense(10, input_dim=16, activation = 'relu'))
        model.add(Dense(4, activation='sigmoid'))
        model.compile(optimizer='rmsprop',
                  loss='mse')
        model.summary()
        print('neural net created')
        return model
    
    def _transform_state_to_vector(self, number):
        """
        Vectorizes an integer number (one-hot encoding).
        
        :param: number: integer the state of the game 
        :returns: np.array: A vector of shape (1, 16) with 0 for most of its values except the position of the provided number
        """
        assert number <= 16 and number >=0
        input_vector = np.zeros((1, 16))
        input_vector[:,number] = 1
        return input_vector
    
    def train(self, total_episodes, epsilon=1.0, max_epsilon=1.0, min_epsilon=0.01, decay_rate=0.01):
        """
        Trains the agent using the basic RL implementation where the Q-table is replaced by a Neural network.
        Improvements such as experience replay are not implemented in this version.
        
        :param: total_episodes: integer number of episdes to train for
        :param: epsilon: float a value between 0 and 1 that describes the probability of random actions by the agent (exploration)
        :param: max_epsilon: the max value for epsilon. Default value is 1.0.
        """
        rewards = []
        for episode in range(total_episodes):
            state = env.reset()
            step = 0
            done = False
            total_rewards = 0
#             print('in epsode', episode, 'eps is', epsilon)
            while True:
                # choose an action based on Q table (step 3)
                # here we implement the exploration possibility in the beginning we want to take random actions
                # so the agent can explore the environment, later once it has learned about the environment it can 
                # start to take actions it knows
                input_vector= self._transform_state_to_vector(state)
                predicted = self.q.predict(input_vector)
                action = np.argmax(predicted)
                if np.random.uniform(high=1, low=0) < epsilon:
                    action = env.action_space.sample()
                
                
                # take the action and observe the outcome state and reward (step 4)        
                next_state, reward, done, info = env.step(action)
                
                # retrain the neural net (step 5)
                input_next_state = self._transform_state_to_vector(next_state)    
                predicted_next_state = self.q.predict(input_next_state)
                Qmax = np.max(predicted_next_state)
                target = predicted
                print('target', target, target.shape)
                target[0][action] = reward + self.gamma * Qmax
                self.q.fit(input_vector, target)
                state = next_state
                total_rewards += reward

                if done == True:
                    break    

            #adjust epsilon by an exponential function with lower bound min_epsilon
            epsilon = min_epsilon + (max_epsilon-min_epsilon)*np.exp(-decay_rate*episode) 
            rewards.append(total_rewards)    
        print ("Score over time: " +  str(sum(rewards)/total_episodes))
    
    def play(self, episodes, max_steps = 1000, render=False):
        """
        Plays the game in self.env. Default implementation is that the agent plays until it succeeds in finishing
        the game.
        
        :param: episodes: integer the number of times the agent should play. 
        """
        for episode in range(episodes):
            state = env.reset()
            step = 0
            done = False
            score = 0
            print("****************************************************")
            print("EPISODE ", episode)
            while score != 1:
                if render:
                    env.render()
                else:
                    print('agent in pos', state)
                # take the action with the maximum expected reward
                action = np.argmax(self.q.predict(self._transform_state_to_vector(state)))
                next_state, reward, done, info = env.step(action)
                step += 1
                score += reward
                if done and reward == 0:
                    print('agent died in step', step, ' traveling from state', state, 'to', next_state)
                    next_state = env.reset()
                    step = 0
                elif reward == 1:
                    print('agent reached finish in step', step, ' traveling from state', state, 'to', next_state)
                state = next_state
        

### Training and playing

In [128]:
# Parameters
nr_of_episodes = 1000
alpha = 0.75 # learning rate
gamma = 0.95 #discount rate (i.e. how much of future actions do we need to take into consideration)

#The exploration/exploitation parameters
epsilon = 1
max_epsilon = 1
min_epsilon = 0.01
decay_rate = 0.01

env = gym.make("FrozenLake-v0")
# env = gym.make("FrozenLake8x8-v0")
agent = Agent(env, alpha, gamma )
agent.train(nr_of_episodes, epsilon, max_epsilon, min_epsilon, decay_rate)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_40 (Dense)             (None, 4)                 44        
Total params: 214
Trainable params: 214
Non-trainable params: 0
_________________________________________________________________
neural net created
target [[ 0.52947956  0.422703    0.49921611  0.44959489]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 160ms/step - loss: 7.2344e-04
target [[ 0.48272067  0.47862759  0.50200629  0.44626132]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 999us/step - loss: 8.3521e-05
target [[ 0.52873945  0.42917037  0.49782658  0.45221996]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.7473e-04
target [[ 0.52445406  0.43000999  0.4990029   0.45067361]] (1, 4)
Epoch 1/1
1/1

target [[ 0.49779674  0.44273144  0.49888134  0.42061391]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 4.1980e-05
target [[ 0.50996011  0.45884249  0.48336431  0.4702898 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.0297e-05
target [[ 0.48389822  0.48881283  0.50174665  0.45910266]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 2.7576e-04
target [[ 0.51703483  0.4782491   0.54949975  0.47006306]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.7500e-04
target [[ 0.51679492  0.47858921  0.54890722  0.47150299]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 5.0904e-07
target [[ 0.4948169   0.47808349  0.45622697  0.43258032]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 982us/step - loss: 0.0010
target [[ 0.50983584  0.45890498  0.48536366  0.47149917]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step

target [[ 0.48737729  0.45158365  0.50321144  0.42521021]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.7513e-04
target [[ 0.48724428  0.45409492  0.50292319  0.42652872]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.3436e-05
target [[ 0.49665904  0.47362006  0.47799906  0.48008606]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.7057e-05
target [[ 0.4961434   0.47335905  0.47878563  0.47841778]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 9.4202e-05
target [[ 0.48649251  0.45401898  0.50214195  0.42609102]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0015
target [[ 0.55862337  0.47726113  0.47259358  0.52731985]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 2.8432e-06
target [[ 0.55882579  0.47725457  0.47237548  0.52793729]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step -

target [[ 0.51400828  0.48162538  0.4997617   0.49056622]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 9.9574e-05
target [[ 0.48915443  0.47779882  0.47841984  0.48036224]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.4955e-04
target [[ 0.4862155   0.47853711  0.47932345  0.47894147]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.4775e-04
target [[ 0.48381847  0.47914052  0.48004732  0.47778696]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.2078e-05
target [[ 0.48704857  0.46250319  0.49707112  0.44022602]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.5251e-06
target [[ 0.48404035  0.47750297  0.47999701  0.477671  ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 7.8010e-05
target [[ 0.48432875  0.47474825  0.48070732  0.47727829]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/st

target [[ 0.47850627  0.47190225  0.47053275  0.48518914]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.4713e-04
target [[ 0.4772512   0.47143176  0.47197568  0.4815743 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 2.0747e-07
target [[ 0.47870183  0.49773598  0.49781328  0.47996876]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.5489e-04
target [[ 0.47902417  0.49764791  0.49570152  0.48053971]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 2.8717e-05
target [[ 0.47732642  0.47101438  0.47132885  0.48171461]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 9.6998e-05
target [[ 0.47477409  0.47133955  0.47200826  0.48064935]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.4439e-04
target [[ 0.47366971  0.47093952  0.47321603  0.47739744]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/st

...

target [[ 0.30850509  0.32503113  0.30304763  0.3090865 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 4.4114e-04
target [[ 0.27881479  0.30404833  0.2758885   0.2889353 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 5.7778e-05
target [[ 0.27837804  0.30244654  0.27567902  0.28829741]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.8376e-05
target [[ 0.30916998  0.32763898  0.30389968  0.3108505 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 8.9960e-07
target [[ 0.33561817  0.34285221  0.30057916  0.34038007]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 7.3467e-05
target [[ 0.33571282  0.34140891  0.29997176  0.3399545 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 9.6796e-06
target [[ 0.36532775  0.35757124  0.36591178  0.36035553]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/st

target [[ 0.27954942  0.30440956  0.27537307  0.28962934]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1000us/step - loss: 2.0492e-05
target [[ 0.31069687  0.33029011  0.30373192  0.31372058]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.8182e-05
target [[ 0.31039643  0.32906744  0.30341864  0.31300527]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 3.6897e-04
target [[ 0.27999774  0.30654153  0.27557516  0.29032925]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 5.8730e-05
target [[ 0.27959698  0.30430874  0.27554488  0.28954259]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 2.5901e-05
target [[ 0.31096748  0.33139148  0.30401826  0.31440672]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 3.3023e-04
target [[ 0.28017259  0.30691639  0.27587315  0.29035744]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms

target [[ 0.27913392  0.30531338  0.27598435  0.29002455]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 5.5966e-08
target [[ 0.30492157  0.32188833  0.2883662   0.30462423]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 2.5217e-04
target [[ 0.27917436  0.30502218  0.27571994  0.29021391]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.1121e-09
target [[ 0.30465606  0.3210049   0.28760105  0.3044138 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.2129e-04
target [[ 0.30950066  0.31462583  0.30703312  0.30773753]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 2.6199e-05
target [[ 0.30448326  0.32036602  0.28709966  0.30427924]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 5.6038e-04
target [[ 0.27909166  0.30479392  0.2757709   0.2900655 ]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/st

target [[ 0.33331987  0.35034457  0.30064139  0.34183359]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.6713e-05
target [[ 0.33311036  0.34893951  0.30039334  0.34094667]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.8683e-05
target [[ 0.27898553  0.31071901  0.27543718  0.29121879]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 4.0964e-05
target [[ 0.31079113  0.34092045  0.30427992  0.31871441]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 7.2642e-05
target [[ 0.31043434  0.33962756  0.30388764  0.31779319]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 7.2092e-05
target [[ 0.31008634  0.33836159  0.30350545  0.31690782]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 7.1555e-05
target [[ 0.30974522  0.33711463  0.30313095  0.31605065]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 2ms/st

target [[ 0.30472678  0.32720169  0.28771317  0.30619863]] (1, 4)
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 4.1463e-04
Score over time: 0.031


In [129]:
agent.play(1)

****************************************************
EPISODE  0
agent in pos 0
agent in pos 4
agent in pos 4
agent in pos 8
agent died in step 4  traveling from state 8 to 12
agent in pos 0
agent in pos 1
agent died in step 2  traveling from state 1 to 5
agent in pos 0
agent in pos 4
agent in pos 8
agent died in step 3  traveling from state 8 to 12
agent in pos 0
agent in pos 1
agent in pos 0
agent in pos 4
agent died in step 4  traveling from state 4 to 5
agent in pos 0
agent in pos 0
agent in pos 1
agent in pos 2
agent in pos 6
agent died in step 5  traveling from state 6 to 7
agent in pos 0
agent in pos 0
agent in pos 4
agent in pos 8
agent in pos 8
agent died in step 5  traveling from state 8 to 12
agent in pos 0
agent in pos 1
agent died in step 2  traveling from state 1 to 5
agent in pos 0
agent in pos 0
agent in pos 1
agent died in step 3  traveling from state 1 to 5
agent in pos 0
agent in pos 4
agent in pos 8
agent died in step 3  traveling from state 8 to 12
agent in pos 0
ag